# Transforming Data
There are 100 values of DRG Definition. Construct 100 DRG Charges features, one for
each unique value of DRG Definition. The feature should record the Average Covered
Charges for the specified DRG category. Then construct a transformed version of the data
that only includes the provider id, provider state, and the 100 new DRG Charges features.
For example, the data should look like the format in the table below. Make sure to include
missing values for any provider that doesn’t have a charge for a specific DRG.

In [ ]:
import pandas as pd

df = pd.read_csv('data.csv')
df[' Average Covered Charges '] = df[' Average Covered Charges '].str[1:].astype(float)
pv = df.pivot(index=['Provider Id', 'Provider State'], 
              values=' Average Covered Charges ', 
              columns='DRG Definition')
pv = pv.reset_index()
pv.head(2)

# Correlation

1. Find the Highest and lowest Correlations

In [ ]:
#pv = pv.fillna(value=0.0)

def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=2):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

def get_bottom_abs_correlations(df, n=2):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=True)
    return au_corr[0:n]

print(get_top_abs_correlations(pv))
print(get_bottom_abs_correlations(pv))

# Scatterplots

2. Plot a scatterplot to show their relationship. </br>
Make sure to label both axis of the plot with the feature names. </br>
Discuss whether the observed relationsare interesting or expected, given the DRG category names. </br>
(This will result in 4
scatter plots total.)

In [ ]:
import plotly.express as px

fig = px.scatter(pv, 
                 x="194 - SIMPLE PNEUMONIA & PLEURISY W CC", 
                 y="690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC")
fig.show()


In [ ]:
fig = px.scatter(pv, 
                 x="392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC", 
                 y="690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC")
fig.show()

In [ ]:
fig = px.scatter(pv, 
                 x="460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC", 
                 y="885 - PSYCHOSES")
fig.show()

In [ ]:
fig = px.scatter(pv, 
                 x="473 - CERVICAL SPINAL FUSION W/O CC/MCC", 
                 y="885 - PSYCHOSES")
fig.show()

In [ ]:
pv['Provider State'].unique()
# CA, TX, NY, FL, WA, KY

In [ ]:
pv = pv[pv['Provider State'].isin(['CA', 'TX', 'NY', 'FL', 'WA', 'KY'])]

In [ ]:
group = pv.groupby('Provider State').mean().reset_index()
group

In [ ]:
import plotly.express as px

fig = px.box(pv, x="Provider State", y="064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC", points='all')
fig.show()

In [ ]:
fig = px.box(pv, x="Provider State", y="853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC", points='all')
fig.show()

In [ ]:
fig = px.box(pv, x="Provider State", y="917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC", points='all')
fig.show()

Based on the three box plots, identify the pair of states that you think have the
most significant differences in their charges for a single DRG category. Explicitly
state your hypothesis in terms of H0 and H1.

<p>H1 California and Kentucky have significant differences in 917 Charges</p>
<p>H0 California and Kentucky have no significant differences in 917 Charges</p>

In [ ]:
from scipy.stats import ttest_ind

cat1 = pv[pv['Provider State'] == 'CA'].dropna(subset=['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC', "853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC", "064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC"]).head(16)
print(len(cat1.index))
cat2 = pv[pv['Provider State'] == 'KY'].dropna(subset=['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC', "853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC", "064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC"])
print(len(cat2.index))
ttest_ind(cat1[['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC', "853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC", "064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC"]], cat2[['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC', "853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC", "064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC"]])

In [ ]:
from scipy.stats import ttest_rel


cat1 = pv[pv['Provider State'] == 'CA'].dropna(subset=['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC', "853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC", "064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC"]).head(16)
print(len(cat1.index))
cat2 = pv[pv['Provider State'] == 'KY'].dropna(subset=['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC', "853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC", "064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC"])
print(len(cat2.index))
ttest_rel(cat1[['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC', "853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC", "064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC"]], cat2[['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC', "853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC", "064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC"]])

In [ ]:
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp

cat1 = pv[pv['Provider State'] == 'CA'].dropna(subset=['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC'])
cat2 = pv[pv['Provider State'] == 'KY'].dropna(subset=['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC'])

ttest_1samp(cat1['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC'], float(cat2['917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC'].mean()))